# Data Cleaning Notebook
----

## 1. Init & Config

Load all libraries at the very beginning.

In [13]:
import pandas as pd
import numpy as np
import pyarrow
import fastparquet
import sys 
sys.path.append('..')
import config


pd.set_option('display.max_columns', None)

## 2. Data Ingestion (Load)

Read the raw file.

In [14]:
df = pd.read_csv(config.RAW_NETFLIX_FILE)

## 3. Initial Audit

Before touching anything, see what you have.


In [15]:
df.head(4)

,show_id,type,title,director,country,date_added,release_year,rating,duration,listed_in
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,United States,9/25/2021,2020,PG-13,90 min,Documentaries
1,s3,TV Show,Ganglands,Julien Leclercq,France,9/24/2021,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act..."
2,s6,TV Show,Midnight Mass,Mike Flanagan,United States,9/24/2021,2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries"
3,s14,Movie,Confessions of an Invisible Girl,Bruno Garotti,Brazil,9/22/2021,2021,TV-PG,91 min,"Children & Family Movies, Comedies"


In [16]:
df['director'] = df['director'].replace('Not Given', 'Unknown')

## 4. Cleaning (The Meat)

Process divided into logical sections.

In [17]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 8790 entries, 0 to 8789
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   show_id       8790 non-null   str  
 1   type          8790 non-null   str  
 2   title         8790 non-null   str  
 3   director      8790 non-null   str  
 4   country       8790 non-null   str  
 5   date_added    8790 non-null   str  
 6   release_year  8790 non-null   int64
 7   rating        8790 non-null   str  
 8   duration      8790 non-null   str  
 9   listed_in     8790 non-null   str  
dtypes: int64(1), str(9)
memory usage: 1.5 MB


#### 4.1. Type Conversion (Cast)
Fix incorrect types first. We cast what we can immediately.

In [18]:

df['date_added'] = pd.to_datetime(df['date_added'])

df = df.astype({
    'show_id': 'object',
    'type': 'category',
    'title': 'object',
    'director':'object',
    'country': 'category',
    'rating': 'category'
})

#### 4.2. Duration Fix (Divide & Conquer)
Separating the mixed 'duration' column into 'duration_minutes' and 'duration_seasons'.

In [19]:
# Extract number and unit using Regex
# (\d+) -> Capture numeric digits
# (\s+) -> Space
# (\w+) -> Capture words (min, Season, Seasons)
df[['amount','unit']] = df['duration'].str.extract(r'(\d+)\s+(\w+)')

# Convert amount to numeric
df['amount'] = pd.to_numeric(df['amount'])

# Create new specific columns
df['duration_minutes'] = np.where(df['unit'] == 'min', df['amount'], np.nan)
df['duration_seasons'] = np.where(df['unit'].str.contains('Season'), df['amount'], np.nan)

# Cast the new seasons column to float (consistent with your original astype plan)
df['duration_seasons'] = df['duration_seasons'].astype('float64')

df = df.drop(columns=['duration', 'amount', 'unit'], errors='ignore')

df[['duration_minutes', 'duration_seasons']].describe()

,duration_minutes,duration_seasons
count,6126.000000,2664.000000
mean,99.584884,1.751877
std,28.283225,1.550622
min,3.000000,1.000000
25%,87.000000,1.000000
50%,98.000000,1.000000
75%,114.000000,2.000000
max,312.000000,17.000000


#### 4.3. List Handling
Splitting comma-separated lists into actual Python lists.

In [ ]:
df['genre_list'] = df['listed_in'].str.split(', ')
df = df.drop(columns=['listed_in'], errors='ignore')
df['genre_list'].head()

0                                      [Documentaries]
1    [Crime TV Shows, International TV Shows, TV Ac...
2                 [TV Dramas, TV Horror, TV Mysteries]
3                 [Children & Family Movies, Comedies]
4    [Dramas, Independent Movies, International Mov...
Name: genre_list, dtype: object

#### 4.4. Null Handling (Imputation)
Handling missing values.

In [21]:

df['director'] = df['director'].fillna('Unknown')

print('--- [Conteo de valores nulos] ---')
print(df.isna().sum())

--- [Conteo de valores nulos] ---
show_id                0
type                   0
title                  0
director               0
country                0
date_added             0
release_year           0
rating                 0
listed_in              0
duration_minutes    2664
duration_seasons    6126
genre_list             0
dtype: int64


## 5. Final Audit

Review the final state of the dataframe.

In [22]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 8790 entries, 0 to 8789
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   show_id           8790 non-null   object        
 1   type              8790 non-null   category      
 2   title             8790 non-null   object        
 3   director          8790 non-null   object        
 4   country           8790 non-null   category      
 5   date_added        8790 non-null   datetime64[us]
 6   release_year      8790 non-null   int64         
 7   rating            8790 non-null   category      
 8   listed_in         8790 non-null   str           
 9   duration_minutes  6126 non-null   float64       
 10  duration_seasons  2664 non-null   float64       
 11  genre_list        8790 non-null   object        
dtypes: category(3), datetime64[us](1), float64(2), int64(1), object(4), str(1)
memory usage: 932.4+ KB


In [23]:
print('[--- [Numero de filas y numeros de columnas] ---\n')
print(f'filas: {df.shape[0]} \ncolumnas: {df.shape[1]}\n')
print('--- [Conteo de filas duplicadas] ---\n')
print(f'{df.duplicated(subset=['show_id']).sum()}')

[--- [Numero de filas y numeros de columnas] ---

filas: 8790 
columnas: 12

--- [Conteo de filas duplicadas] ---

0



## 6. Export
Save the clean result to avoid running everything again.

In [24]:
df.to_parquet('../data/processed/netflix_cleaned.parquet')